In [1]:
!nvcc --version


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [2]:
code= """
#include <stdio.h>
#include <stdlib.h>

#define N 1000000

__global__ void add(int *a, int *b, int *c) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if (tid < N) {
        c[tid] = a[tid] + b[tid];
    }
}

int main() {
    int *a, *b, *c;
    int *d_a, *d_b, *d_c;
    int size = N * sizeof(int);

    // Allocate memory on host
    a = (int*)malloc(size);
    b = (int*)malloc(size);
    c = (int*)malloc(size);

    // Initialize arrays
    for (int i = 0; i < N; i++) {
        a[i] = i;
        b[i] = i * 2;
    }

    // Allocate memory on device
    cudaMalloc(&d_a, size);
    cudaMalloc(&d_b, size);
    cudaMalloc(&d_c, size);

    // Copy data from host to device
    cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);

    // Launch kernel with 1 million threads
    add<<<(N + 255) / 256, 256>>>(d_a, d_b, d_c);

    // Copy result from device to host
    cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);

    // Print first and last elements of result
    printf("c[0]=%d, c[%d] = %d",c[0],N-1,c[N-1]);

    // Free memory
    free(a);
    free(b);
    free(c);
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

    return 0;
}
"""


In [3]:
text_file=open("assign4.cu","w")
text_file.write(code)
text_file.close()


In [4]:
!nvcc assign4.cu

In [5]:
!./a.out

c[0]=0, c[999999] = 2999997

In [6]:
!nvprof ./a.out

==1694== NVPROF is profiling process 1694, command: ./a.out
==1694== Profiling application: ./a.out
c[0]=0, c[999999] = 2999997==1694== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   50.85%  1.5456ms         1  1.5456ms  1.5456ms  1.5456ms  [CUDA memcpy DtoH]
                   47.59%  1.4467ms         2  723.33us  684.66us  762.00us  [CUDA memcpy HtoD]
                    1.56%  47.294us         1  47.294us  47.294us  47.294us  add(int*, int*, int*)
      API calls:   97.27%  230.11ms         3  76.702ms  70.977us  229.96ms  cudaMalloc
                    2.00%  4.7387ms         3  1.5796ms  930.78us  2.8221ms  cudaMemcpy
                    0.40%  952.51us         1  952.51us  952.51us  952.51us  cuDeviceGetPCIBusId
                    0.24%  568.88us         3  189.63us  156.25us  206.96us  cudaFree
                    0.05%  126.66us       101  1.2540us     144ns  53.591us  cuDeviceGetAttribute
                